# Optional Instructions

**To run locally in your browser (if downloaded):**
* install voila:  
`pip install voila`
* open a terminal in the location of where you're keeping this jupyter notebook     

* trust the notebook if not done already:  
`jupyter trust "probability calculator functions.ipynb"`
* Voila blocks file access by default. serve your working directory:  
`voila --Voila.root_dir=.` then click on the notebook file.  

* **If you dont want to do all that and just want to run it normally,** just open this up in your Jupyter notebook IDE of choice and use the gui at the bottom of the notebook (**"-----GUI-----"** section).  
* if you dont want a GUI, or want examples of how to run the functions directly, see the other notebook.

**NOTE:**  
 Make sure to run the entire notebook and all of its functions for this to work. (VS code and other jupyter notebook IDEs have an "execute above cells" button.)  
Also, you can compact code cells so that they dont take up so much screen space.

**OTHER NOTE:** if the GUI isn't showing up when you run the cell, find this snippet of code in the GUI code box and comment it out:


In [17]:
# # Display everything in the notebook
# display(HTML("""
# <style>
# .output_wrapper, .output {
#     height: 800px !important;
# }
# .output_scroll {
#     height: 800px !important;
#     max-height: 800px !important;
# }
# </style>
# """))

# -----Backend calculations & plotting-----

In [3]:
# libraries for calculators and visualization 
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.stats import binom, geom, norm
from typing import List, Union, Tuple
import random, itertools


In [4]:
# Permutations and Combinations
def calculate_permutation(n: int, r: int) -> int:
    """
    Calculate permutation (nPr) - order matters
    
    Args:
        n (int): Total number of items
        r (int): Number of items being chosen
    
    Returns:
        int: Number of possible permutations
    """
    if n < r:
        raise ValueError("n must be greater than or equal to r")
    return math.factorial(n) // math.factorial(n - r)

def calculate_combination(n: int, r: int) -> int:
    """
    Calculate combination (nCr) - order doesn't matter
    
    Args:
        n (int): Total number of items
        r (int): Number of items being chosen
    
    Returns:
        int: Number of possible combinations
    """
    if n < r:
        raise ValueError("n must be greater than or equal to r")
    return math.factorial(n) // (math.factorial(r) * math.factorial(n - r))


In [5]:
## Simple Bayes
def bayesian_probability(prior: float, likelihood: float, evidence: float) -> float:
    """
    Calculate posterior probability using Bayes' theorem
    P(A|B) = P(B|A) * P(A) / P(B)
    
    Args:
        prior (float): P(A) - Prior probability
        likelihood (float): P(B|A) - Likelihood
        evidence (float): P(B) - Evidence
    
    Returns:
        float: Posterior probability P(A|B)
    """
    if not all(0 <= x <= 1 for x in [prior, likelihood, evidence]):
        raise ValueError("All probabilities must be between 0 and 1")
    if evidence == 0:
        raise ValueError("Evidence probability cannot be zero")
    
    posterior = (likelihood * prior) / evidence
    return posterior

In [6]:
## Bayes probability visualization
def bayes_theorem(prior_probability, likelihood, evidence):
    """
    Calculate posterior probability using Bayes' Theorem
    
    Parameters:
    prior_probability (float): P(A), the prior probability of hypothesis A
    likelihood (float): P(B|A), the likelihood of evidence B given hypothesis A
    evidence (float): P(B), the total probability of evidence B
    
    Returns:
    float: P(A|B), the posterior probability of hypothesis A given evidence B
    """
    posterior_probability = (prior_probability * likelihood) / evidence
    return posterior_probability


def calculate_evidence(prior_probability, likelihood, alternative_likelihood):
    """
    Calculate the total probability of evidence (denominator in Bayes' Theorem)
    
    Parameters:
    prior_probability (float): P(A), the prior probability of hypothesis A
    likelihood (float): P(B|A), the likelihood of evidence B given hypothesis A
    alternative_likelihood (float): P(B|¬A), the likelihood of evidence B given not hypothesis A
    
    Returns:
    float: P(B), the total probability of evidence B
    """
    # P(B) = P(B|A) * P(A) + P(B|¬A) * P(¬A)
    evidence = (likelihood * prior_probability) + (alternative_likelihood * (1 - prior_probability))
    return evidence


def visualize_bayes(prior_prob, true_positive_rate, false_positive_rate, population=10000):
    """
    Visualize Bayes' theorem with proportional small boxes.
    
    Parameters:
      prior_prob (float): Prior probability of the condition.
      true_positive_rate (float): P(Test+|Condition+).
      false_positive_rate (float): P(Test+|Condition-).
      population (int): Total population for visualization.
    """
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches

    # Calculate populations and test outcomes
    cond_pos = int(prior_prob * population)
    cond_neg = population - cond_pos
    tp = int(true_positive_rate * cond_pos)
    fp = int(false_positive_rate * cond_neg)
    total_pos = tp + fp
    posterior = tp / total_pos if total_pos else 0

    fig, ax = plt.subplots(figsize=(14, 10))
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 100)
    ax.set_aspect('equal')
    ax.axis('off')

    # Draw the big box (x=0 to 100, y=40 to 90; height=50)
    prior_x = prior_prob * 100  # width for condition+ (left side)
    ax.add_patch(patches.Rectangle((0, 40), 100, 50, linewidth=2, edgecolor='black', facecolor='none'))
    ax.plot([prior_x, prior_x], [40, 90], 'k-', linewidth=2)
    ax.add_patch(patches.Rectangle((0, 40), prior_x, 50, facecolor='lightblue', alpha=0.5))
    ax.add_patch(patches.Rectangle((prior_x, 40), 100 - prior_x, 50, facecolor='lightgreen', alpha=0.5))
    
    # Add title at the top
    ax.text(50, 95, "Probability (Population)",
            ha='center', va='center', fontsize=14, fontweight='bold')
    
    # Add labels for the condition
    ax.text(prior_x/2, 85, "Condition +", ha='center', va='bottom', fontsize=12)
    ax.text(prior_x + (100 - prior_x)/2, 85, "Condition -", ha='center', va='bottom', fontsize=12)
    
    # Add counts directly under the labels
    ax.text(prior_x/2, 80, f"{cond_pos} / {population}\n({prior_prob:.1%})",
            ha='center', va='center', fontsize=12)
    ax.text(prior_x + (100 - prior_x)/2, 80, f"{cond_neg} / {population}\n({1 - prior_prob:.1%})",
            ha='center', va='center', fontsize=12)

    # Compute areas of the big box sides
    A_left_big = prior_x * 50           # condition+ side area
    A_right_big = (100 - prior_x) * 50    # condition- side area
    # Desired areas for small boxes
    A_blue = true_positive_rate * A_left_big   # blue box area (TP)
    A_red  = false_positive_rate * A_right_big  # red box area (FP)

    # Available widths on each side
    blue_avail, red_avail = prior_x, 100 - prior_x
    H_default = 20

    # Compute candidate widths with default height
    blue_candidate = A_blue / H_default if H_default else 0
    red_candidate = A_red / H_default if H_default else 0

    # Adjust heights if candidate widths exceed available width
    blue_height = H_default if blue_candidate <= blue_avail else A_blue / blue_avail
    red_height = H_default if red_candidate <= red_avail else A_red / red_avail

    # Attempt to use a common height if possible
    H_common = max(blue_height, red_height)
    if (A_blue / H_common <= blue_avail) and (A_red / H_common <= red_avail):
        blue_height = red_height = H_common
        blue_width = A_blue / H_common
        red_width = A_red / H_common
    else:
        blue_width = A_blue / blue_height
        red_width = A_red / red_height

    # For the blue box: if its computed width is smaller than the left side,
    # shift it right so its right edge touches the big box divider.
    blue_box_x = (prior_x - blue_width) if blue_width < blue_avail else 0
    
    # Center the boxes vertically in the main box
    main_box_center_y = 65  # Center of the main box (40 + 50/2)
    blue_box_y = main_box_center_y - blue_height/2
    red_box_y = main_box_center_y - red_height/2
    
    # Draw the blue box (TP)
    ax.add_patch(patches.Rectangle((blue_box_x, blue_box_y), blue_width, blue_height,
                                   linewidth=2, edgecolor='black', facecolor='darkblue', alpha=0.6))
    ax.text(blue_box_x + blue_width/2, blue_box_y + blue_height/2,
            f"TP: {true_positive_rate:.0%}*{cond_pos} = {tp}\n({tp/population:.2%} of total)",
            ha='center', va='center', fontsize=10, color='white')

    # Draw the red box on the right side (position fixed at prior_x)
    red_box_x = prior_x
    ax.add_patch(patches.Rectangle((red_box_x, red_box_y), red_width, red_height,
                                   linewidth=2, edgecolor='black', facecolor='darkred', alpha=0.6))
    ax.text(red_box_x + red_width/2, red_box_y + red_height/2,
            f"FP: {false_positive_rate:.0%}*{cond_neg} = {fp}\n({fp/population:.2%} of total)",
            ha='center', va='center', fontsize=10, color='white')

    # Draw the Positive Test Results box at the bottom
    tp_prop = tp / total_pos if total_pos else 0
    fp_prop = fp / total_pos if total_pos else 0
    box_x, box_y, box_w, box_h = 10, 10, 80, 20
    ax.text(box_x + box_w/2, box_y + box_h + 5,
            f"Positive Test Results (Total: {total_pos})",
            ha='center', va='center', fontsize=14, fontweight='bold')
    ax.add_patch(patches.Rectangle((box_x, box_y), box_w, box_h,
                                   linewidth=2, edgecolor='black', facecolor='none'))
    divider_x = box_x + tp_prop * box_w
    ax.plot([divider_x, divider_x], [box_y, box_y + box_h], 'k-', linewidth=2)
    ax.add_patch(patches.Rectangle((box_x, box_y), tp_prop * box_w, box_h,
                                   facecolor='darkblue', alpha=0.6))
    ax.add_patch(patches.Rectangle((divider_x, box_y), fp_prop * box_w, box_h,
                                   facecolor='darkred', alpha=0.6))
    ax.text(box_x + (tp_prop * box_w)/2, box_y + box_h/2,
            f"True Positives: {tp}\n({tp_prop:.2%})",
            ha='center', va='center', fontsize=10, color='white')
    ax.text(divider_x + (fp_prop * box_w)/2, box_y + box_h/2,
            f"False Positives: {fp}\n({fp_prop:.2%})",
            ha='center', va='center', fontsize=10, color='white')

    # Draw arrows from the bottom of small boxes to the Positive Test Results box
    arrow_props = dict(arrowstyle='->', linewidth=2, color='purple')
    ax.annotate('', xy=(box_x + tp_prop * box_w/2, box_y + box_h),
                xytext=(blue_box_x + blue_width/2, blue_box_y),  
                arrowprops=arrow_props)
    ax.annotate('', xy=(divider_x + fp_prop * box_w/2, box_y + box_h),
                xytext=(red_box_x + red_width/2, red_box_y),  
                arrowprops=arrow_props)

    # Display Bayes' theorem result
    result_text = (f"Bayes' Theorem Result:\n"
                   f"P(Condition+|Test+) = {posterior:.4f} = {posterior:.2%}\n"
                   f"P(Condition-|Test+) = {1 - posterior:.4f} = {1 - posterior:.2%}")
    ax.text(50, 0, result_text, ha='center', va='bottom', fontsize=12, fontweight='bold',
            bbox=dict(facecolor='yellow', alpha=0.2))

    plt.tight_layout()
    plt.show()
    return posterior

In [7]:
# Binomial Probability
def binomial_probability(n: int, p: float, k: int) -> Tuple[float, None]:
    """
    Calculate binomial probability and optionally create visualization
    P(X = k) = C(n,k) * p^k * (1-p)^(n-k)
    
    Args:
        n (int): Number of trials
        p (float): Probability of success on each trial
        k (int): Number of successes
    
    Returns:
        Tuple[float, None]: (probability, None)
    """
    if not 0 <= p <= 1:
        raise ValueError("Probability must be between 0 and 1")
    if k > n:
        raise ValueError("Number of successes cannot exceed number of trials")
    
    probability = binom.pmf(k, n, p)
    return probability


In [8]:
# Binomial prob visualization
def plot_binomial_distribution(n: int, p: float) -> None:
    """
    Create visualization of binomial probability distribution
    
    Args:
        n (int): Number of trials
        p (float): Probability of success on each trial
    """
    k = np.arange(0, n + 1)
    probabilities = binom.pmf(k, n, p)
    
    plt.figure(figsize=(10, 6))
    plt.bar(k, probabilities, alpha=0.8, color='blue')
    plt.title(f'Binomial Distribution (n={n}, p={p})')
    plt.xlabel('Number of Successes (k)')
    plt.ylabel('Probability')
    plt.grid(True, alpha=0.3)
    plt.show()

In [9]:
# Bernoulli process
def bernoulli_process(p: float, n: int) -> np.ndarray:
    """
    Simulate a Bernoulli process (sequence of Bernoulli trials)
    
    Args:
        p (float): Probability of success for each trial
        n (int): Number of trials to simulate
        
    Returns:
        np.ndarray: Array of 0s and 1s representing the outcomes
    """
    if not 0 <= p <= 1:
        raise ValueError("Probability must be between 0 and 1")
    if n <= 0:
        raise ValueError("Number of trials must be positive")
        
    # Vectorized implementation for efficiency
    return np.random.random(n) < p

def visualize_bernoulli_process(outcomes: np.ndarray, p: float):
    """
    Visualize a Bernoulli process
    
    Args:
        outcomes (np.ndarray): Array of 0s and 1s representing trial outcomes
        p (float): Probability of success used in the simulation
    """
    n = len(outcomes)
    
    # Print summary statistics
    print(f"Bernoulli Process Summary (p={p}, n={n}):")
    print(f"Expected successes: {p*n:.2f}")
    print(f"Observed successes: {np.sum(outcomes)}")
    print(f"Expected success rate: {p:.4f}")
    print(f"Observed success rate: {np.mean(outcomes):.4f}")


    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Plot 1: Cumulative successes
    cumulative_successes = np.cumsum(outcomes)
    expected_cumulative = np.arange(1, n+1) * p
    
    ax1.plot(range(1, n+1), cumulative_successes, 'b-', label='Observed successes')
    ax1.plot(range(1, n+1), expected_cumulative, 'r--', label=f'Expected (p={p})')
    ax1.set_title('Cumulative Successes in Bernoulli Process')
    ax1.set_xlabel('Number of Trials')
    ax1.set_ylabel('Number of Successes')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Success rate convergence
    success_rates = np.zeros(n)
    for i in range(n):
        if i == 0:
            success_rates[i] = outcomes[i]
        else:
            success_rates[i] = np.mean(outcomes[:i+1])
    
    ax2.plot(range(1, n+1), success_rates, 'g-', label='Observed success rate')
    ax2.axhline(y=p, color='r', linestyle='--', label=f'True probability (p={p})')
    ax2.set_title('Convergence of Success Rate to True Probability')
    ax2.set_xlabel('Number of Trials')
    ax2.set_ylabel('Success Rate')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()



In [10]:
# Geometric Probability
def geometric_probability(p: float, k: int) -> Tuple[float, None]:
    """
    Calculate geometric probability and optionally create visualization
    P(X = k) = p * (1-p)^(k-1)
    
    Args:
        p (float): Probability of success
        k (int): Number of trials until first success
    
    Returns:
        Tuple[float, None]: (probability, None)
    """
    if not 0 <= p <= 1:
        raise ValueError("Probability must be between 0 and 1")
    if k < 1:
        raise ValueError("Number of trials must be at least 1")
    
    probability = geom.pmf(k, p)
    return probability

def plot_geometric_distribution(p: float, max_k: int = 20) -> None:
    """
    Create visualization of geometric probability distribution
    
    Args:
        p (float): Probability of success
        max_k (int): Maximum number of trials to plot
    """
    k = np.arange(1, max_k + 1)
    probabilities = geom.pmf(k, p)
    
    plt.figure(figsize=(10, 6))
    plt.bar(k, probabilities, alpha=0.8, color='green')
    plt.title(f'Geometric Distribution (p={p})')
    plt.xlabel('Number of Trials Until First Success (k)')
    plt.ylabel('Probability')
    plt.grid(True, alpha=0.3)
    plt.show()

In [11]:
# Probability Density Function (PDF) of normal distribution
def plot_normal_distribution(mean: float, std_dev: float, value: float) -> float:
    """
    Plot a normal distribution and show probability of finding a specific value
    
    Args:
        mean (float): Mean of the distribution
        std_dev (float): Standard deviation of the distribution
        value (float): Value to find probability for
    
    Returns:
        float: Probability density at the specified value
    """
    # Create points for the bell curve
    x = np.linspace(mean - 4*std_dev, mean + 4*std_dev, 1000)
    y = norm.pdf(x, mean, std_dev)
    
    # Calculate probability density at the specified value
    prob_density = norm.pdf(value, mean, std_dev)
    
    # Create the plot
    plt.figure(figsize=(10, 6))
    
    # Plot the bell curve
    plt.plot(x, y, 'b-', label='Normal Distribution')
    
    # Fill the area up to the value
    x_fill = x[x <= value]
    y_fill = norm.pdf(x_fill, mean, std_dev)
    plt.fill_between(x_fill, y_fill, alpha=0.3, color='blue')
    
    # Mark the specific value
    plt.plot([value], [prob_density], 'ro', label=f'Value: {value}')
    
    # Add vertical line at the value
    plt.axvline(x=value, color='r', linestyle='--', alpha=0.3)
    
    # Calculate cumulative probability
    cumulative_prob = norm.cdf(value, mean, std_dev)
    
    # Add labels and title
    plt.title(f'Normal Distribution (μ={mean}, σ={std_dev})')
    plt.xlabel('Value')
    plt.ylabel('Probability Density')
    
    # Add text box with probabilities
    text = f'Value: {value:.2f}\n'
    text += f'Probability Density: {prob_density:.4f}\n'
    text += f'Cumulative Probability: {cumulative_prob:.2%}'
    plt.text(0.95, 0.95, text,
             transform=plt.gca().transAxes,
             verticalalignment='top',
             horizontalalignment='right',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    return prob_density

In [12]:
# P-value
def calculate_p_value(mean: float, std_dev: float, observed_value: float, two_tailed: bool = True) -> tuple:
    """
    Calculate and visualize p-value for given parameters
    
    Args:
        mean (float): Null hypothesis mean
        std_dev (float): Standard deviation
        observed_value (float): Observed test statistic
        two_tailed (bool): If True, calculates two-tailed p-value
    
    Returns:
        tuple: (p_value, plot)
    """
    # Calculate z-score
    z_score = (observed_value - mean) / std_dev
    
    # Calculate p-value
    if two_tailed:
        p_value = 2 * (1 - norm.cdf(abs(z_score)))
    else:
        # One-tailed: use left or right tail based on observed value
        if observed_value > mean:
            p_value = 1 - norm.cdf(z_score)  # Right-tailed
        else:
            p_value = norm.cdf(z_score)  # Left-tailed
    
    # Create visualization
    x = np.linspace(mean - 4*std_dev, mean + 4*std_dev, 1000)
    y = norm.pdf(x, mean, std_dev)
    
    plt.figure(figsize=(12, 6))
    
    # Plot the normal distribution
    plt.plot(x, y, 'b-', label='Normal Distribution')
    
    # Shade p-value region(s)
    if two_tailed:
        # Shade both tails
        critical_value = abs(observed_value - mean)
        x_left = x[x <= mean - critical_value]
        x_right = x[x >= mean + critical_value]
        plt.fill_between(x_left, norm.pdf(x_left, mean, std_dev), 
                        color='red', alpha=0.3, label='p-value region')
        plt.fill_between(x_right, norm.pdf(x_right, mean, std_dev), 
                        color='red', alpha=0.3)
    else:
        # Shade one tail
        if observed_value > mean:
            x_tail = x[x >= observed_value]
            plt.fill_between(x_tail, norm.pdf(x_tail, mean, std_dev), 
                           color='red', alpha=0.3, label='p-value region')
        else:
            x_tail = x[x <= observed_value]
            plt.fill_between(x_tail, norm.pdf(x_tail, mean, std_dev), 
                           color='red', alpha=0.3, label='p-value region')
    
    # Add vertical line at observed value
    plt.axvline(x=observed_value, color='r', linestyle='--', 
                label=f'Observed Value: {observed_value}')
    
    # Add text box with statistics
    text = f'Observed Value: {observed_value:.2f}\n'
    text += f'Mean (H₀): {mean:.2f}\n'
    text += f'Standard Deviation: {std_dev:.2f}\n'
    text += f'Z-score: {z_score:.2f}\n'
    text += f'p-value: {p_value:.4f}\n'
    text += f'Test Type: {"Two-tailed" if two_tailed else "One-tailed"}'
    
    plt.text(0.95, 0.95, text,
             transform=plt.gca().transAxes,
             verticalalignment='top',
             horizontalalignment='right',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.title('P-value Visualization')
    plt.xlabel('Value')
    plt.ylabel('Probability Density')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    return p_value


In [13]:
# Card game probability
# Build a standard 52-card deck.
def get_deck():
    return [r+s for r in "23456789TJQKA" for s in "cdhs"]

# Convert card's rank to numeric value.
def card_value(card):
    return "23456789TJQKA".index(card[0]) + 2

# Evaluate a 5-card hand and return a tuple (hand_category, tie-breakers).
def evaluate_five(cards):
    r = sorted([card_value(c) for c in cards], reverse=True)
    s = [c[1] for c in cards]
    flush = len(set(s)) == 1
    sr = sorted(r)
    straight = False; high_straight = None
    if len(set(r)) == 5:
        if sr[-1] - sr[0] == 4:
            straight, high_straight = True, sr[-1]
        elif sr == [2, 3, 4, 5, 14]:
            straight, high_straight = True, 5
    counts = {}
    for v in r:
        counts[v] = counts.get(v, 0) + 1
    items = sorted(counts.items(), key=lambda x: (x[1], x[0]), reverse=True)
    # Categories: 9=Straight Flush, 8=Four-of-a-Kind, 7=Full House, 6=Flush,
    # 5=Straight, 4=Three-of-a-Kind, 3=Two Pair, 2=One Pair, 1=High Card.
    if flush and straight:
        return (9, high_straight)
    if items[0][1] == 4:
        four = items[0][0]
        kicker = [v for v in r if v != four][0]
        return (8, four, kicker)
    if items[0][1] == 3 and len(counts) == 2:
        return (7, items[0][0], items[1][0])
    if flush:
        return (6,) + tuple(r)
    if straight:
        return (5, high_straight)
    if items[0][1] == 3:
        triple = items[0][0]
        kickers = sorted([v for v in r if v != triple], reverse=True)
        return (4, triple) + tuple(kickers)
    if len(counts) == 3 and items[0][1] == 2 and items[1][1] == 2:
        high_pair = max(items[0][0], items[1][0])
        low_pair = min(items[0][0], items[1][0])
        kicker = [v for v in r if v != high_pair and v != low_pair][0]
        return (3, high_pair, low_pair, kicker)
    if len(counts) == 4 and items[0][1] == 2:
        pair = items[0][0]
        kickers = sorted([v for v in r if v != pair], reverse=True)
        return (2, pair) + tuple(kickers)
    return (1,) + tuple(r)

# For more than 5 cards, determine the best 5-card hand.
def best_hand(cards):
    return max((evaluate_five(combo) for combo in itertools.combinations(cards, 5)), default=None)

# Run the simulation.
def simulate(your_cards, opp_known, num_opponents, community_cards, sims=10000):
    num_hole = len(your_cards)
    results = {i: 0 for i in range(1, 10)}  # your hand categories counts.
    opp_better = 0
    opp_known_better = {opp: 0 for opp in opp_known} if opp_known else {}
    opp_known_results = {opp: {cat: 0 for cat in range(1, 10)} for opp in opp_known} if opp_known else {}
    for _ in range(sims):
        deck = get_deck()
        used = your_cards + community_cards
        for cards in opp_known.values():
            used += cards
        deck = [c for c in deck if c not in used]
        random.shuffle(deck)
        comm = community_cards.copy()
        while len(comm) < 5:
            comm.append(deck.pop())
        opp_hands = []
        for i in range(1, num_opponents + 1):
            if opp_known and i in opp_known:
                opp_hands.append(opp_known[i])
            else:
                opp_hands.append([deck.pop() for _ in range(num_hole)])
        your_best = best_hand(your_cards + comm)
        results[your_best[0]] += 1
        any_better = False
        for i, opp in enumerate(opp_hands, start=1):
            opp_best = best_hand(opp + comm)
            if opp_known and i in opp_known:
                opp_known_results[i][opp_best[0]] += 1
            if opp_best > your_best:
                any_better = True
                if opp_known and i in opp_known:
                    opp_known_better[i] += 1
        if any_better:
            opp_better += 1
    prob = {k: (v / sims * 100) for k, v in results.items()}
    opp_prob = opp_better / sims * 100
    opp_known_prob = {k: (v / sims * 100) for k, v in opp_known_better.items()} if opp_known else {}
    opp_known_dist = {opp: {cat: (cnt / sims * 100) for cat, cnt in dist.items()} for opp, dist in opp_known_results.items()} if opp_known else {}
    return prob, opp_prob, opp_known_prob, opp_known_dist

# Map hand category to name.
def hand_name(cat):
    names = {1: "High Card", 2: "One Pair", 3: "Two Pair", 4: "Three of a Kind",
             5: "Straight", 6: "Flush", 7: "Full House", 8: "Four of a Kind",
             9: "Straight Flush"}
    return names.get(cat, "Unknown")


# -----GUI-----

In [16]:
import ipywidgets as widgets
from IPython.display import HTML, display

# Create output container for dynamic page switching
output = widgets.Output()

# Function to switch pages
def show_page(content):
    with output:
        output.clear_output()
        display(content)

# Function to create back button and page layout
def back_to_main(_):
    show_page(main_menu)

# Function to create a page layout with a back button and box with border and shadow
def create_page(title, content=None):
    back_button = widgets.Button(description="Back", button_style="primary")
    back_button.on_click(back_to_main)
    page_content = [widgets.HTML(f"<h2>{title}</h2>")]
    
    if content:
        page_content.append(content)
        
    return widgets.VBox(
        page_content + [back_button],
        layout=widgets.Layout(align_items="center", width="80%", height="auto", border="2px solid black", box_shadow="2px 2px 10px rgba(0, 0, 0, 0.3)", padding="10px", margin="20px")
    )

# Pages
# 1️⃣ Permutations & Combinations Page
def create_permutations_combinations_page():
    # Input boxes for Permutations & Combinations
    n_input = widgets.IntText(description="n (Total):", layout=widgets.Layout(width="250px"))
    r_input = widgets.IntText(description="r (Selections):", layout=widgets.Layout(width="250px"))
    
    # Permutations Box
    perm_box = widgets.VBox([
        widgets.HTML("<h3>Permutations</h3>"),
        widgets.HTML("Formula: P(n, r) = n! / (n - r)!"),
        n_input, r_input
    ])
    
    # Combinations Box
    comb_box = widgets.VBox([
        widgets.HTML("<h3>Combinations</h3>"),
        widgets.HTML("Formula: C(n, r) = n! / (r!(n - r)!"),
        n_input, r_input
    ])
    
    # Permutations Button
    perm_button = widgets.Button(description="Calculate Permutations", layout=widgets.Layout(width="200px", height="40px"))
    result_permutations = widgets.Output()
    def on_perm_button_clicked(b):
        with result_permutations:
            result_permutations.clear_output()
            n_perm = calculate_permutation(n_input.value, r_input.value)
            print(f"Permutation (n={n_input.value}, r={r_input.value}): {n_perm}")
    
    # Combinations Button
    comb_button = widgets.Button(description="Calculate Combinations", layout=widgets.Layout(width="200px", height="40px"))
    result_combinations = widgets.Output()
    def on_comb_button_clicked(b):
        with result_combinations:
            result_combinations.clear_output()
            n_comb = calculate_combination(n_input.value, r_input.value)
            print(f"Combination (n={n_input.value}, r={r_input.value}): {n_comb}")
    
    perm_button.on_click(on_perm_button_clicked)
    comb_button.on_click(on_comb_button_clicked)
    
    return create_page("Permutations & Combinations", widgets.VBox([
        perm_box, perm_button, result_permutations, comb_box, comb_button, result_combinations
    ]))


# Probability Density Function (PDF) Page
def create_pdf_page():
    # Input boxes for Normal Distribution
    mean_input = widgets.FloatText(description="Mean:", layout=widgets.Layout(width="250px"))
    std_dev_input = widgets.FloatText(description="Std Dev:", layout=widgets.Layout(width="250px"))
    value_input = widgets.FloatText(description="Value:", layout=widgets.Layout(width="250px"))
    
    # PDF Button
    pdf_button = widgets.Button(description="Calculate PDF", layout=widgets.Layout(width="200px", height="40px"))
    result_pdf = widgets.Output()
    def on_pdf_button_clicked(b):
        with result_pdf:
            result_pdf.clear_output()
            probability = plot_normal_distribution(mean_input.value, std_dev_input.value, value_input.value)  # Assuming this function exists
            print(f"\nProbability density at {value_input.value}: {probability:.4f}")
    
    pdf_button.on_click(on_pdf_button_clicked)
    
    return create_page("Probability Density Function", widgets.VBox([
        mean_input, std_dev_input, value_input, pdf_button, result_pdf
    ]))

# 2️⃣ Binomial Probability Page
def create_binomial_probability_page():
    # Input boxes for Binomial Probability
    n_input = widgets.IntText(description="n (Trials):", layout=widgets.Layout(width="250px"))
    p_input = widgets.FloatText(description="p (Probability):", layout=widgets.Layout(width="250px"))
    k_input = widgets.IntText(description="k (Successes):", layout=widgets.Layout(width="250px"))
    
    # Binomial Probability Button
    binom_button = widgets.Button(description="Calculate Binomial Probability", layout=widgets.Layout(width="200px", height="40px"))
    result_binom = widgets.Output()
    def on_binom_button_clicked(b):
        with result_binom:
            result_binom.clear_output()
            bin_prob = binomial_probability(n_input.value, p_input.value, k_input.value)  # Use your function
            print(f"Binomial Probability (n={n_input.value}, p={p_input.value}, k={k_input.value}): {bin_prob:.2f}")
            print("\nDistribution plot:")
            plot_binomial_distribution(n_input.value, p_input.value)  # Plotting function
    
    binom_button.on_click(on_binom_button_clicked)
    
    return create_page("Binomial Probability", widgets.VBox([
        n_input, p_input, k_input, binom_button, result_binom
    ]))

# 3️⃣ Geometric Probability Page
def create_geometric_probability_page():
    # Input boxes for Geometric Probability
    p_input = widgets.FloatText(description="p (Probability):", layout=widgets.Layout(width="250px"))
    k_input = widgets.IntText(description="k (Trials):", layout=widgets.Layout(width="250px"))
    
    # Geometric Probability Button
    geom_button = widgets.Button(description="Calculate Geometric Probability", layout=widgets.Layout(width="200px", height="40px"))
    result_geom = widgets.Output()
    def on_geom_button_clicked(b):
        with result_geom:
            result_geom.clear_output()
            geom_prob = geometric_probability(p_input.value, k_input.value)  # Use your function
            print(f"Geometric Probability (p={p_input.value}, k={k_input.value}): {geom_prob:.2f}")
            print("\nDistribution plot:")
            plot_geometric_distribution(p_input.value)  # Plotting function
    
    geom_button.on_click(on_geom_button_clicked)
    
    return create_page("Geometric Probability", widgets.VBox([
        p_input, k_input, geom_button, result_geom
    ]))

# 4️⃣ P-Value Page
def create_p_value_page():
    # Input boxes for P-Value
    mean_input = widgets.FloatText(description="Mean:", layout=widgets.Layout(width="250px"))
    std_dev_input = widgets.FloatText(description="Std Dev:", layout=widgets.Layout(width="250px"))
    observed_value_input = widgets.FloatText(description="Observed Value:", layout=widgets.Layout(width="250px"))
    two_tailed_input = widgets.Checkbox(description="Two-tailed?", value=True, layout=widgets.Layout(width="250px"))
    
    # P-Value Button
    p_value_button = widgets.Button(description="Calculate P-Value", layout=widgets.Layout(width="200px", height="40px"))
    result_p_value = widgets.Output()
    def on_p_value_button_clicked(b):
        with result_p_value:
            result_p_value.clear_output()
            p_value_two = calculate_p_value(mean_input.value, std_dev_input.value, observed_value_input.value, two_tailed_input.value)  # Use your function
            print(f"Two-tailed p-value: {p_value_two:.4f}")
            print(f"(% chance to get a result just like that or even more extreme: {p_value_two*100:.2f}%)")
    
    p_value_button.on_click(on_p_value_button_clicked)
    
    return create_page("P-Value", widgets.VBox([
        mean_input, std_dev_input, observed_value_input, two_tailed_input, p_value_button, result_p_value
    ]))

# 5️⃣ Bernoulli Trials Page
def create_bernoulli_process_page():
    # Input boxes for Bernoulli process
    p_input = widgets.FloatText(description="p (Probability of success):", layout=widgets.Layout(width="250px"))
    n_input = widgets.IntText(description="n (Trials):", layout=widgets.Layout(width="250px"))
    
    # Bernoulli process Button
    bernoulli_button = widgets.Button(description="Simulate Bernoulli Trials", layout=widgets.Layout(width="200px", height="40px"))
    result_bernoulli = widgets.Output()
    def on_bernoulli_button_clicked(b):
        with result_bernoulli:
            result_bernoulli.clear_output()
            outcomes = bernoulli_process(p_input.value, n_input.value)  # Use your function
            visualize_bernoulli_process(outcomes, p_input.value)  # Visualization function
    
    bernoulli_button.on_click(on_bernoulli_button_clicked)
    
    return create_page("Bernoulli Trials", widgets.VBox([
        p_input, n_input, bernoulli_button, result_bernoulli
    ]))

# 7️⃣ Bayes Probability Page
def create_bayes_probability_page():
    # 📌 Description Section
    description = widgets.HTML("""
    <h2>Bayes Probability Visualization explained</h2>
    <p>Ex: You have a population of 10,000 that's 23% infected. A new virus test has an 85% TP rate and a 20% FP rate. You can use this to visualize:</p>
    <ul>
        <li>The probability a tester is infected, given they've tested positive: P(sick | tested positive) -> <i>P(Condition + | Test +)</i></li>
        <li>The probability a tester is not infected, given they've tested positive: P(not sick | tested positive) -> <i>P(Condition - | Test +)</i></li>
        <li>The number of people in the full population that's expected to TP: P(TP in population) -> 25% * 2500 = 625</li>
        <li>The number of people in the population that's expected to FP: P(FP in population) -> 25% * 2500 = 625</li>
        <li>The percentage of the full population that tests TP: P(Condition+ ⋂ Test+) -> (x% of total)</li>
        <li>The percentage of the full population that tests FP: P(Condition- ⋂ Test+) -> (y% of total)</li>
    </ul>
    <p><strong>Note:</strong> The big box represents the population, with the left side being the sick side (Condition +) and the right side being the non-sick side (Condition -).</p>
    <p><i>Based on a method by Avanti Sethi (UT Dallas):</i> <a href="https://www.youtube.com/watch?v=piCJ70jFvYE">Bayes' Theorem Visualization</a></p>
    <hr>
    """)

    # Common layout for input boxes
    input_layout = widgets.Layout(width="300px")
    label_layout = widgets.Layout(width="200px")  # Ensures labels aren't cut off

    # 📌 Simple Bayes Calculator Section
    calculator_title = widgets.HTML("<h3>Simple Bayes Calculator</h3>")
    calculator_subtitle = widgets.HTML("<p><i>Calculate posterior probability using Bayes' theorem:</i></p><h4>P(A|B) = [P(B|A) × P(A)] / P(B)</h4>")

    prior_input = widgets.FloatText(description="Prior probability P(A):", value=0.23, layout=input_layout, style={"description_width": "initial"})
    likelihood_input = widgets.FloatText(description="Likelihood P(B|A):", value=0.85, layout=input_layout, style={"description_width": "initial"})
    evidence_input = widgets.FloatText(description="Evidence P(B):", value=0.40, layout=input_layout, style={"description_width": "initial"})

    calc_button = widgets.Button(description="Calculate Posterior", layout=widgets.Layout(width="250px", height="40px"))
    calc_result = widgets.Output()

    def on_calc_button_clicked(b):
        with calc_result:
            calc_result.clear_output()
            prior = prior_input.value
            likelihood = likelihood_input.value
            evidence = evidence_input.value
            posterior = bayes_theorem(prior, likelihood, evidence)
            print(f"Posterior probability P(A|B): {posterior:.4f}")

    calc_button.on_click(on_calc_button_clicked)

    calculator_box = widgets.VBox([
        calculator_title,
        calculator_subtitle,
        prior_input, likelihood_input, evidence_input,
        calc_button, calc_result
    ], layout=widgets.Layout(border="1px solid black", padding="10px", width="100%"))

    # 📌 Bayes Visualizer Section
    visualizer_title = widgets.HTML("<h3>Bayes Visualizer</h3>")

    pop_input = widgets.IntText(description="Population Size:", value=10000, layout=input_layout, style={"description_width": "initial"})
    prior_viz_input = widgets.FloatText(description="Prior probability P(A):", value=0.23, layout=input_layout, style={"description_width": "initial"})
    tp_rate_viz_input = widgets.FloatText(description="True Positive Rate P(B|A):", value=0.85, layout=input_layout, style={"description_width": "initial"})
    fp_rate_viz_input = widgets.FloatText(description="False Positive Rate P(B|¬A):", value=0.20, layout=input_layout, style={"description_width": "initial"})

    viz_button = widgets.Button(description="Visualize Bayes", layout=widgets.Layout(width="250px", height="40px"))
    viz_result = widgets.Output()

    def on_viz_button_clicked(b):
        with viz_result:
            viz_result.clear_output()
            prior = prior_viz_input.value
            tp_rate = tp_rate_viz_input.value
            fp_rate = fp_rate_viz_input.value
            population = pop_input.value
            visualize_bayes(prior, tp_rate, fp_rate, population)
            print("Bayes Visualization Complete.")

    viz_button.on_click(on_viz_button_clicked)

    visualizer_box = widgets.VBox([
        visualizer_title,
        pop_input, prior_viz_input, tp_rate_viz_input, fp_rate_viz_input,
        viz_button, viz_result
    ], layout=widgets.Layout(border="1px solid black", padding="10px", width="100%"))

    # 📌 Full Page Layout
    page_content = widgets.VBox([
        description,
        calculator_box,
        visualizer_box
    ])
    
    return create_page("Bayes Probability", page_content)

# 8️⃣ Card Games Page
def create_card_games_page():
    title = widgets.HTML("""<h1>Card Games</h1>
        <p>Determines hand probabilities using the Monte Carlo method (determining the chance of something happening by running a computer algorithm a bunch of times and counting the number of 'successes').</p>""")
    
    # Checkbox for Omaha Poker
    omaha_checkbox = widgets.Checkbox(description="Click here for Omaha Poker (4 card hands)")
    
    # Input for number of simulations
    sim_input = widgets.IntText(value=10000, description="Simulations:", layout=widgets.Layout(width="250px"))
    
    # Input for number of players (2-10)
    players_input = widgets.BoundedIntText(value=2, min=2, max=10, description="Number of players:", layout=widgets.Layout(width="250px"))
    
    # Reset Button
    reset_button = widgets.Button(description="Reset", layout=widgets.Layout(width="250px", height="40px"))
    
    # Calculate Probability Button
    calc_button = widgets.Button(description="Calculate Probability", layout=widgets.Layout(width="250px", height="40px"))
    result_output = widgets.Output()
    
    # Function to create a card widget
    def create_card():
        button = widgets.Button(description="", layout=widgets.Layout(width="60px", height="90px"))
        button.card_value = None
        button.style.button_color = 'lightgray'
        
        card_back_html = """
        <div style="width:100%; height:100%; display:flex; justify-content:center; align-items:center; background-color:#f8f8f8; border:2px solid #888; border-radius:5px;">
            <div style="width:80%; height:80%; background-color:#d00; border-radius:5px; display:flex; justify-content:center; align-items:center;">
                <div style="color:white; font-weight:bold; font-size:20px;">♠♥♣♦</div>
            </div>
        </div>
        """
        card_html = widgets.HTML(card_back_html)
        
        def on_button_click(b):
            rank_options = [str(i) for i in range(2, 11)] + list("JQKA")
            rank = widgets.Dropdown(options=rank_options, description="Rank")
            
            suit_options = [
                ("♥", "Hearts"), 
                ("♦", "Diamonds"), 
                ("♣", "Clubs"), 
                ("♠", "Spades")
            ]
            suit = widgets.Dropdown(options=suit_options, description="Suit")
            
            confirm = widgets.Button(description="Confirm")
            cancel = widgets.Button(description="Cancel")
            card_selection_interface.children = [rank, suit, widgets.HBox([confirm, cancel])]
            
            def on_confirm(b):
                rank_val = rank.value
                suit_val = suit.value
                
                if isinstance(suit_val, tuple):
                    suit_symbol, suit_name = suit_val
                else:
                    for symbol, name in suit_options:
                        if name == suit_val:
                            suit_symbol = symbol
                            suit_name = name
                            break
                    else:
                        suit_symbol = suit_val
                        suit_name = suit_val
                
                button.card_value = f"{rank_val}{suit_symbol}"
                button.style.button_color = '#90EE90'  # Light green
                
                if suit_name in ["Hearts", "Diamonds"]:
                    button.style.font_color = 'red'
                else:
                    button.style.font_color = 'black'
                
                button.description = f"{rank_val}{suit_symbol}"
                card_selection_interface.children = []  # Clear the selection interface
            
            def on_cancel(b):
                card_selection_interface.children = []  # Clear the selection interface
                
            confirm.on_click(on_confirm)
            cancel.on_click(on_cancel)
        
        button.on_click(on_button_click)
        return button
    
    # Community Cards (5 cards, top left)
    community_label = widgets.HTML("<h4>Community Cards</h4>")
    community_cards = widgets.HBox([create_card() for _ in range(5)], layout=widgets.Layout(margin="10px 0"))
    
    # Player's hand (top right, labeled "YOUR CARDS")
    player_label = widgets.HTML("<h3>YOUR CARDS</h3>")
    player_cards = widgets.HBox([create_card() for _ in range(2)])
    
    # Update function for player's hand based on game type
    def update_player_hand():
        num_cards = 4 if omaha_checkbox.value else 2
        player_cards.children = [create_card() for _ in range(num_cards)]
    
    # Watch for changes in game type
    def on_omaha_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            update_player_hand()
            update_opponents()
    
    omaha_checkbox.observe(on_omaha_change)
    
    # Opponents' hands (in a 5x2 grid)
    opponent_container = widgets.GridBox(
        [], 
        layout=widgets.Layout(
            grid_template_columns="repeat(2, 1fr)",
            grid_gap="15px", 
            width="100%",
            padding="10px"
        )
    )
    
    def update_opponents():
        num_players = players_input.value
        num_opponents = num_players - 1
        num_cards = 4 if omaha_checkbox.value else 2
        
        opponent_boxes = []
        for i in range(num_opponents):
            label = widgets.HTML(f"<h4>Player {i+2}'s Hand</h4>")
            hand = widgets.HBox([create_card() for _ in range(num_cards)], layout=widgets.Layout(padding="5px"))
            opponent_box = widgets.VBox([label, hand], layout=widgets.Layout(border="1px solid #ccc", padding="5px"))
            opponent_boxes.append(opponent_box)
        
        opponent_container.children = opponent_boxes
    
    # Observer for player count changes
    def on_players_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            update_opponents()
    
    players_input.observe(on_players_change)
    
    # Initialize the opponent display
    update_opponents()
    
    # Handler for calculate button
    def on_calculate(b):
        with result_output:
            result_output.clear_output()
            print("Running simulation...")
            
            # Gather all selected cards
            your_cards = [card.card_value for card in player_cards.children if card.card_value]
            community = [card.card_value for card in community_cards.children if card.card_value]
            
            # Gather known opponent cards
            opp_known = {}
            for i, box in enumerate(opponent_container.children):
                hand = box.children[1]  # The HBox containing the cards
                cards = [card.card_value for card in hand.children if card.card_value]
                if cards:  # Only include opponents with known cards
                    opp_known[i + 1] = cards
            
            # Number of opponents
            num_opponents = players_input.value - 1  # Excluding the main player
            
            # Run simulation
            prob, opp_prob, opp_known_prob, opp_known_dist = simulate(your_cards, opp_known, num_opponents, community, sim_input.value)
            
            # Display results
            print("\nYour Hand Probabilities:")
            for cat in range(1, 10):
                print(f"{hand_name(cat)}: {prob[cat]:.2f}%")
            
            if opp_known_dist:
                for opp, dist in opp_known_dist.items():
                    print(f"\nOpponent {opp} (known cards) hand probabilities:")
                    for cat in range(1, 10):
                        print(f"{hand_name(cat)}: {dist[cat]:.2f}%")
            
            print(f"\nChance an opponent has a better hand: {opp_prob:.2f}%")
            
            if opp_known_prob:
                for opp, p in opp_known_prob.items():
                    print(f"Opponent {opp} (known cards)'s chance of a better hand: {p:.2f}%")
    
    calc_button.on_click(on_calculate)
    
    # Handler for reset button
    def on_reset(b):
        # Reset all inputs and outputs
        sim_input.value = 10000
        players_input.value = 2
        omaha_checkbox.value = False
        result_output.clear_output()
        update_player_hand()
        update_opponents()
    
    reset_button.on_click(on_reset)
    
    # Layout adjustments
    community_section = widgets.VBox([
        community_label,
        community_cards
    ], layout=widgets.Layout(width="50%", padding="10px", border="1px solid #ccc"))
    
    player_section = widgets.VBox([
        player_label,
        player_cards,
        calc_button  # Moved the calculate button here
    ], layout=widgets.Layout(width="50%", padding="10px", border="1px solid #ccc", margin_left="20px"))
    
    playing_field = widgets.HBox([
        community_section,
        player_section
    ], layout=widgets.Layout(justify_content="space-between", width="100%"))
    
    # Card selection interface at the bottom of the page
    card_selection_interface = widgets.HBox([], layout=widgets.Layout(justify_content="center", margin="20px 0"))
    
    return create_page("Card Games", widgets.VBox([
        title, 
        widgets.HBox([
            widgets.VBox([
                omaha_checkbox,
                sim_input, 
                players_input, 
                reset_button  # Added reset button here
            ]),
        ], layout=widgets.Layout(justify_content="space-between")),
        card_selection_interface,  # Add the card selection interface here
        widgets.HTML("<h3>Playing Field</h3>"),
        playing_field,
        widgets.HTML("<h3>Other Players</h3>"),
        opponent_container,

        result_output
    ]))

# 6️⃣ Simple Box for Other Pages
def create_simple_page(title):
    return create_page(title, widgets.VBox([
        widgets.HTML(f"<h3>WIP. Coming soon!</h3>")
    ], layout=widgets.Layout(width="300px", padding="10px", border="2px solid black", box_shadow="2px 2px 10px rgba(0, 0, 0, 0.3)")))


# Define pages
pages = {
    "Permutations & Combinations": create_permutations_combinations_page(),
    "Bayes Probability": create_bayes_probability_page(),
    "Binomial Probability": create_binomial_probability_page(),
    "Geometric Probability": create_geometric_probability_page(),
    "Probability Density Function": create_pdf_page(),
    "P-Value": create_p_value_page(),
    "Bernoulli Process": create_bernoulli_process_page(),
    "Card Games": create_card_games_page(),
    "AI Chat": create_simple_page("AI Chat"),
}

# Function to create a button tile
def create_tile(name, icon_filename, description):
    icon_path = f"static/{icon_filename}"  # Ensure icons are in "static" folder (for viola)

    with open(icon_path, "r") as f:
        svg_content = f.read()

    # Set fixed size for SVG
    svg_display = widgets.HTML(f'<div style="width:100px; height:100px; overflow:hidden;">{svg_content}</div>')

    button = widgets.Button(description=name, layout=widgets.Layout(width="150px", height="40px"))
    button.on_click(lambda b: show_page(pages[name]))

    return widgets.VBox(
        [svg_display, button, widgets.HTML(f"<p style='text-align:center; font-size:12px;'>{description}</p>")],
        layout=widgets.Layout(align_items="center", width="350px", height="200px", border="2px solid black", box_shadow="2px 2px 10px rgba(0, 0, 0, 0.3)")
    )

# List of menu items (name, icon file, description)
menu_items = [
    ("Permutations & Combinations", "permutations-combinations.svg", "Calculate the number of possible arrangements and selections."),
    ("Bayes Probability", "bayes-probability.svg", "Calculate and visualize Bayes probability."),
    ("Binomial Probability", "binomial-probability.svg", "Calculate the chance of getting exactly k successes."),
    ("Bernoulli Process", "bernoulli-trials.svg", "Calculate & graph cumulative success rate over n trials."),
    ("Geometric Probability", "geometric-probability.svg", "Determine chance of first success occurring after k trials."),
    ("Probability Density Function", "probability-density-function.svg", "Visualize probability distributions."),
    ("P-Value", "p-value.svg", "Chance of finding your value (or one more extreme than it)."),
    ("Card Games", "playing-card-back.svg", "Monte Carlo sim for Poker."),
    ("AI Chat", "ai-chat.svg", "See if our AI can figure the prob question you've got."),
]


# Create scrollable grid
grid_layout = widgets.GridBox(
    [create_tile(name, icon, desc) for name, icon, desc in menu_items],
    layout=widgets.Layout(grid_template_columns="repeat(3, 1fr)", grid_gap="10px", justify_items="center")
)

# Create creator credit with link and highlighted text
creator_credit = widgets.HTML(
    """
    <div style="text-align: center; margin-top: 20px; margin-bottom: 10px;">
        <a href="https://www.radicool.club" target="_blank" style="text-decoration: none; color: #555;">
            created by Radicool Solutions, LLC <span style="color: #0066cc; text-decoration: underline;">(website)</span>
        </a>
    </div>
    """
)

# Create Buy Me a Coffee button with SVG icon
coffee_button = widgets.HTML(
    """
    <div style="text-align: center; margin-bottom: 20px;">
        <a href="https://buymeacoffee.com/radicoolsolutions" target="_blank" style="text-decoration: none;">
            <button style="
                display: flex;
                align-items: center;
                background-color: #FFDD00;
                color: #000000;
                border: none;
                padding: 8px 16px;
                border-radius: 20px;
                cursor: pointer;
                font-weight: bold;
                box-shadow: 0 2px 5px rgba(0,0,0,0.1);
            ">
                <img src="static/coffee-cup.svg" style="height: 20px; margin-right: 8px;" />
                Buy me a coffee
            </button>
        </a>
    </div>
    """
)

# Display everything in the notebook
display(HTML("""
<style>
.output_wrapper, .output {
    height: 800px !important;
}
.output_scroll {
    height: 800px !important;
    max-height: 800px !important;
}
</style>
"""))

# Define main menu with added elements
main_menu = widgets.VBox(
    [
        widgets.HTML("<h1>Probability Calculator</h1>"), 
        grid_layout,
        creator_credit,
        coffee_button
    ],
    layout=widgets.Layout(overflow_y="auto", max_height="500px", align_items="center")
)

# Show main menu on startup
show_page(main_menu)

display(output)

Output()